In [1]:
import pandas as pd

In [2]:
df_ = pd.read_json('news.json')

In [3]:
df = df_[['title','related']]

In [4]:
df.head()

,title,related
0,Corinthians se atrapalha em diagnóstico e tent...,no
1,Cidades da região recebem doses que serão usad...,yes
2,Santa Lúcia confirma caso de raiva bovina,yes
3,Vacina antirrábica é aplicada com agendamento ...,yes
4,Divertida Mente 2: Vá da Raiva à Alegria com o...,no


In [5]:
df.tail()

,title,related
95,VACINAÇÃO CONTRA RAIVA É REALIZADA NO SÁBADO N...,yes
96,Morcegos infectados com raiva são encontrados ...,yes
97,Prefeitura de Montes Claros faz bloqueio contr...,yes
98,Nissan Sentra 2025 foi renovado e mata de raiv...,no
99,Voluntários de resgate ou de abrigos de animai...,yes


In [6]:
df['out'] = df['related'].map({'yes':1,'no':0})

/tmp/ipykernel_870141/2043172176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['out'] = df['related'].map({'yes':1,'no':0})


In [7]:
df

,title,related,out
0,Corinthians se atrapalha em diagnóstico e tent...,no,0
1,Cidades da região recebem doses que serão usad...,yes,1
2,Santa Lúcia confirma caso de raiva bovina,yes,1
3,Vacina antirrábica é aplicada com agendamento ...,yes,1
4,Divertida Mente 2: Vá da Raiva à Alegria com o...,no,0
...,...,...,...
95,VACINAÇÃO CONTRA RAIVA É REALIZADA NO SÁBADO N...,yes,1
96,Morcegos infectados com raiva são encontrados ...,yes,1
97,Prefeitura de Montes Claros faz bloqueio contr...,yes,1
98,Nissan Sentra 2025 foi renovado e mata de raiv...,no,0


In [8]:
from sklearn.preprocessing import LabelEncoder
from keras import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.utils import pad_sequences
import nltk
from nltk.stem.snowball import SnowballStemmer
import regex as re
from nltk.tokenize import sent_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

2024-06-23 09:16:02.871825: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 09:16:02.908050: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 09:16:02.908074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 09:16:02.909008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 09:16:02.914411: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-23 09:16:02.915054: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [9]:
from keras.preprocessing.text import one_hot

In [10]:
text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"
def preprocess_filter(text, stem=False):
    text = re.sub(text_cleaning, " ", str(text.lower()).strip())
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                stemmer = SnowballStemmer(language='portuguese')
                token = stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [11]:
df['title'].size

100

In [13]:
# download some packages
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jpantonow/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jpantonow/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jpantonow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# longest sentence length
def longest_sentence_length(text):
    return len(text.split())
df['maximum_length'] = df['title'].apply(lambda x: longest_sentence_length(x))
print('longest sentence having length -')
max_length = max(df['maximum_length'].values)
print(max_length)

longest sentence having length -
21


In [18]:
size_vocab = df['maximum_length'].sum()

In [19]:
size_vocab

1309

In [20]:
# Word embedding with pre padding
def one_hot_encoded(text, vocab_size=size_vocab, max_length=21):
    hot_encoded = one_hot(text, vocab_size)
    return hot_encoded

In [ ]:
# # Word embedding with pre padding
# def one_hot_encoded(text, vocab_size=5000, max_length=40):
#     hot_encoded = to_categorical(text, vocab_size)
#     return hot_encoded

In [21]:
# word embedding pipeline
def word_embedding(text):
    preprocessed_text = preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

In [22]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedded_features = 21
max_length = 21

model = Sequential()
model.add(Embedding(size_vocab, embedded_features, input_length=max_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 21)            27489     
                                                                 
 lstm (LSTM)                 (None, 10)                1280      
                                                                 
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 28780 (112.42 KB)
Trainable params: 28780 (112.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [23]:
from keras.preprocessing.text import one_hot


In [24]:
one_hot_encoded_title = df['title'].apply(lambda x: word_embedding(x)).values


In [25]:
one_hot_encoded_title

array([list([60, 1187, 707, 1068, 252, 30, 89]),
       list([379, 799, 571, 599, 589, 906, 6, 493, 30, 606, 1276, 1297, 139, 143]),
       list([184, 695, 310, 442, 30, 473]),
       list([27, 472, 360, 469, 140]),
       list([727, 833, 850, 1294, 30, 784, 536, 1183, 173]),
       list([271, 425, 239, 974, 30]),
       list([27, 472, 232, 482, 982, 254]),
       list([822, 621, 758, 621, 199, 1238, 603, 815, 820, 30, 1150, 365, 220]),
       list([113, 1118, 136, 38, 460, 30, 1297, 109]),
       list([819, 131, 669, 27, 1297, 493, 30, 1153, 1164, 978, 388]),
       list([728, 966, 234, 941, 451, 493, 30, 922]),
       list([6, 493, 30, 1017, 1143, 127, 334, 113]),
       list([770, 30, 1203, 1296, 295, 1302]),
       list([154, 38, 807, 30, 67, 1190, 420, 484, 184, 140]),
       list([6, 493, 30, 267, 941, 451]),
       list([1153, 1164, 148, 6, 472, 629, 735]),
       list([30, 743, 1119, 998, 183, 162]),
       list([727, 833, 850, 1294, 30, 784, 536, 1183, 173]),
       list([1208

In [26]:
# padding to make the size equal of the sequences
padded_encoded_title = pad_sequences(
    one_hot_encoded_title, maxlen=max_length, padding='pre')

In [27]:
import numpy as np

In [28]:
# Splitting
X = padded_encoded_title
y = df['out'].values
y = np.array(y)

# shapes
print(X.shape)
print(y.shape)

(100, 21)
(100,)


In [ ]:
X

In [ ]:
y

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

# Shape and size of train and test dataset
print('X train shape {}'.format(X_train.shape))
print('X test shape {}'.format(X_test.shape))
print('y train shape {}'.format(y_train.shape))
print('y test shape {}'.format(y_test.shape))

X train shape (75, 21)
X test shape (25, 21)
y train shape (75,)
y test shape (25,)


In [30]:
history= model.fit(X_train, y_train, validation_data=(
    X_test, y_test), epochs=10, batch_size=4)

Epoch 1/10
19/19 [==============================] - 2s 26ms/step - loss: 0.6770 - accuracy: 0.8533 - val_loss: 0.6476 - val_accuracy: 0.9200
Epoch 2/10
19/19 [==============================] - 0s 6ms/step - loss: 0.6260 - accuracy: 0.8267 - val_loss: 0.5628 - val_accuracy: 0.9200
Epoch 3/10
19/19 [==============================] - 0s 5ms/step - loss: 0.5298 - accuracy: 0.8267 - val_loss: 0.3988 - val_accuracy: 0.9200
Epoch 4/10
19/19 [==============================] - 0s 6ms/step - loss: 0.4339 - accuracy: 0.8267 - val_loss: 0.2975 - val_accuracy: 0.9200
Epoch 5/10
19/19 [==============================] - 0s 6ms/step - loss: 0.3890 - accuracy: 0.8267 - val_loss: 0.2788 - val_accuracy: 0.9200
Epoch 6/10
19/19 [==============================] - 0s 6ms/step - loss: 0.3373 - accuracy: 0.8267 - val_loss: 0.2526 - val_accuracy: 0.9200
Epoch 7/10
19/19 [==============================] - 0s 6ms/step - loss: 0.2769 - accuracy: 0.8267 - val_loss: 0.2054 - val_accuracy: 0.9200
Epoch 8/10
19/19 [=

In [72]:
def predict(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences(
        [encoded], maxlen=max_length, padding='pre')
    output = model.predict(padded_encoded_title)
    print(output)
    output = np.where(output >= 0.85, 1, 0)
    if output[0][0] == 1:
        return 'Relacionado ao tema'
    return 'Não relacionado ao tema'

In [73]:
predict('oi')

1/1 [==============================] - 0s 16ms/step
[[0.6916496]]


'Não relacionado ao tema'

In [74]:
predict('raiva corinthians')

1/1 [==============================] - 0s 16ms/step
[[0.6922663]]


'Não relacionado ao tema'

In [75]:
predict('raiva do caralho')

1/1 [==============================] - 0s 16ms/step
[[0.77842915]]


'Não relacionado ao tema'

In [76]:
predict('Vacina antirrábica é aplicada ')

1/1 [==============================] - 0s 19ms/step
[[0.9097004]]


'Relacionado ao tema'

In [77]:
predict('policial tem surto de raiva')

1/1 [==============================] - 0s 17ms/step
[[0.8266267]]


'Não relacionado ao tema'

In [78]:
predict('Vacina antirrábica é aplicada com agendamento na VISAM')

1/1 [==============================] - 0s 16ms/step
[[0.9483973]]


'Relacionado ao tema'

In [79]:
predict('morcego com raiva é detectado')

1/1 [==============================] - 0s 17ms/step
[[0.85924757]]


'Relacionado ao tema'

In [80]:
predict('Secretaria de Saúde de Indaiatuba vacina animais contra raiva neste sábado no bairro Tombadouro')

1/1 [==============================] - 0s 16ms/step
[[0.9704852]]


'Relacionado ao tema'

In [85]:
predict('Jovem mata mãe em surto de raiva')

1/1 [==============================] - 0s 16ms/step
[[0.6890552]]


'Não relacionado ao tema'